# Function calling parser - Example

Function Calling 让模型能够调用外部工具，来增强自身能力。

In [33]:
import os
from openai import OpenAI
from pydantic import Field

# 这里使用 OpenAI SDK 调用 智谱AI-API
os.environ["OPENAI_API_KEY"] = "***************"
os.environ["OPENAI_BASE_URL"] = "https://open.bigmodel.cn/api/paas/v4/"

client = OpenAI(
    # api_key="YOUR_API_KEY",
    # base_url="BASE_URL"
)

# 示例

这里以获取用户当前位置的天气信息为例，先假设存在一个获取天气的函数或API。

In [34]:
def get_weather(location: str = Field(description="一个城市或地区位置的名称，如上海市、深圳市等")) -> str:
    """获取某个位置的天气，用户应该首先提供一个位置"""
    # 这里可以添加获取天气的逻辑
    return {"location": location, "weather": "sunny"}

## 设置模型可能会调用的工具列表

大部分大语言模型API都会提供参数用于设置模型可能会调用的工具列表。**目前，仅支持 `function` 作为工具。**，要求以 JSON 输入请求参数的 function 列表。下面是一个将方法`get_weather`转为 JSON 的示例。

```python
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "Get weather of an location, the user shoud supply a location first",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    }
                },
                "required": ["location"]
            },
        }
    },
]
```

`Function JSON` 的定义如下：
- type: Literal["function"] = `tool 的类型。目前仅支持 function。`
- function: object
    - name: string = `要调用的 function 名称。必须由 a-z、A-Z、0-9 字符组成，或包含下划线和连字符，最大长度为 64 个字符。`
    - description: string = `function 的功能描述，供模型理解何时以及如何调用该 function。`
    - parameters: object = `function 的输入参数，以 JSON Schema 对象描述。省略 parameters 会定义一个参数列表为空的 function。`

这里提供了提供了一个方法`parse_func_to_json`，可以快速将指定方法转换为 JSON。

In [35]:
import json
from function_calling_parser import parse_func_to_json

parse_param_get_weather = parse_func_to_json(get_weather)

json_text = json.dumps(parse_param_get_weather, ensure_ascii=False, indent=4)

print(f"parse_param_get_weather: \n{parse_param_get_weather}")
print(f"json_text: \n{json_text}")

parse_param_get_weather: 
{'type': 'function', 'function': {'name': 'get_weather', 'description': '一个城市或地区位置的名称，如上海市、深圳市等', 'parameters': {'type': 'object', 'properties': {'location': {'description': '一个城市或地区位置的名称，如上海市、深圳市等', 'title': 'Location', 'type': 'string'}}, 'required': ['location']}, 'strict': False}}
json_text: 
{
    "type": "function",
    "function": {
        "name": "get_weather",
        "description": "一个城市或地区位置的名称，如上海市、深圳市等",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "description": "一个城市或地区位置的名称，如上海市、深圳市等",
                    "title": "Location",
                    "type": "string"
                }
            },
            "required": [
                "location"
            ]
        },
        "strict": false
    }
}


## 调用大模型选择工具
上述代码中`get_weather`的函数功能是需由用户提供的，**因为实际上模型本身并不会执行具体的函数**。

在`OpenAI SDK`中，`Fucntion JSON` 在后续会被实例化为`ChatCompletionToolParam`对象，这个对象包含了工具的类型（目前工具的类型只支持`function`）和方法及其参数的描述信息。

所以本质上`Function Calling`也是一个类似提示的功能，将funtion和参数等描述信息作为提示词，依靠大模型的能力选择能够解决问题的function，并输出确定选择的function和输入参数。

在请求时可以通过设置参数`tools`输入模型可能会调用的 tool 的列表。

In [36]:
tools = [
    parse_param_get_weather
]
messages=[
    {"role": "user", "content": "今天上海市的天气怎么样？出门是否需要带雨伞？"}
]

def send_messages(messages):
    response = client.chat.completions.create(
        model="glm-4-plus",
        messages=messages,
        tools=tools
    )
    return response.choices[0].message

response_message = send_messages(messages)

tool_calls = response_message.tool_calls

print(f"client: \n{response_message}")
print(f"client - tool_calls: \n{tool_calls}")

client: 
ChatCompletionMessage(content=None, refusal=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_9041712753237553585', function=Function(arguments='{"location": "上海市"}', name='get_weather'), type='function', index=0)])
client - tool_calls: 
[ChatCompletionMessageToolCall(id='call_9041712753237553585', function=Function(arguments='{"location": "上海市"}', name='get_weather'), type='function', index=0)]


## 解析并调用模型返回的 function

通过分析打印模型的输出内容，可以发现模型返回的每一个工具调用都被封装成一个`ChatCompletionMessageToolCall`实例，解析它可以得到模型选择的Function和参数信息。

通过实例化一个`ToolCallParser`对象，调用`parse_to_messages`方法可以帮助我们解析模型选择的 Function，将调用的函数信息和函数返回的结果封装成一个角色为`tool`的`message`，返回一个消息列表。

In [37]:
from function_calling_parser import ToolCallParser

# 需要定义一个字典，存储函数名到函数体的映射。
function_map = {
    "get_weather": get_weather
}
# 通过function_map初始化一个ToolCallParser对象
tool_call_parser = ToolCallParser(function_map=function_map)
tool_call_messages = tool_call_parser.parse_to_messages(tool_calls=tool_calls)

print(f"tool_call_messages: {tool_call_messages}")

tool_call_messages: [{'role': 'tool', 'tool_call_id': 'call_9041712753237553585', 'name': 'get_weather', 'content': "{'location': '上海市', 'weather': 'sunny'}"}]


除此之外`ToolCallParser`对象还提供了`parse`和`parse_to_result_contents`方法
- `parse`方法：将解析`tool_calls`并封装成`ToolCall`实例。
- 不同于`ChatCompletionMessageToolCall`实例，`ToolCall`实例封装了`Function`实例，提供了后加载的方法`get_result`和`get_result_content`，用于获取调用工具的执行结果。
- `parse_to_result_contents`方法：获取调用工具的执行结果并强制转换为字符串形式，是一个字符串列表，不同于`parse_to_messages`，没有额外的`role`和`tool_call_id`等信息。

In [38]:

# ToolCall.parse
tool_calls = tool_call_parser.parse(tool_calls=tool_calls)
print(f"tool_calls: \n{tool_calls}")

# 调用ToolCall的实例方法，获取调用工具的执行结果
for tool_call in tool_calls:
    print(f"\ntool-{tool_call.id}-called-result:{tool_call.function.get_result()}\n")

# ToolCall.parse_to_result_contents
tool_call_result_contents = tool_call_parser.parse_to_result_contents(tool_calls=tool_calls)
print(f"tool_calls - results: \n{tool_call_result_contents}")


tool_calls: 
[ToolCall(id='call_9041712753237553585', function=Function(call=<function get_weather at 0x10afab420>, name='get_weather', arguments={'location': '上海市'}))]

tool-call_9041712753237553585-called-result:{'location': '上海市', 'weather': 'sunny'}

tool_calls - results: 
["{'location': '上海市', 'weather': 'sunny'}"]


# 将调用Function的结果传给模型
上文获取的`message`中`content`字段的内容是调用Function得到的结果，可以将`message`作为历史记录添加到上下文中回传给大模型。

In [39]:
# 将调用工具的结果作为聊天消息添加到上下文历史中
messages.extend(tool_call_messages)

response_message = send_messages(messages)
answer = response_message.content

print(f"client: \n{answer}")

client: 
今天上海市的天气是晴朗的，所以出门不需要带雨伞。可以享受阳光明媚的一天！


# 简单查看一下对话历史

In [40]:
# 提取ChatCompletionMessage对象的属性
role = response_message.role
content = response_message.content

messages.append({
    "role": role,
    "content": content
})
print(f"chat_history: \n{json.dumps(messages, ensure_ascii=False, indent=4)}")

chat_history: 
[
    {
        "role": "user",
        "content": "今天上海市的天气怎么样？出门是否需要带雨伞？"
    },
    {
        "role": "tool",
        "tool_call_id": "call_9041712753237553585",
        "name": "get_weather",
        "content": "{'location': '上海市', 'weather': 'sunny'}"
    },
    {
        "role": "assistant",
        "content": "今天上海市的天气是晴朗的，所以出门不需要带雨伞。可以享受阳光明媚的一天！"
    }
]
